In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, StructType, StructField

In [2]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.3.2")
                .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") 
                .config("spark.hadoop.fs.s3a.access.key", "minio") 
                .config("spark.hadoop.fs.s3a.secret.key", "minio123") 
                .config("spark.hadoop.fs.s3a.path.style.access", "true") 
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
                .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
                .config("spark.sql.catalog.spark_catalog.type", "hadoop")
                .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://commerce/warehouse/") 
                .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") 
                .appName("Kafka to MinIO Pipeline")
         .getOrCreate())

25/02/11 11:21:36 WARN Utils: Your hostname, Gauravs-MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.1.27 instead (on interface en0)
25/02/11 11:21:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/saurabh/Library/Caches/pypoetry/virtualenvs/realtime-fraud-detection-system-eUiKKk2v-py3.13/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/saurabh/.ivy2/cache
The jars for the packages stored in: /Users/saurabh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6a0bedf7-1cbf-4cfb-a668-a61d86e69b0f;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in ce

AnalysisException: [INVALID_EXTRACT_BASE_FIELD_TYPE] Can't extract a value from "value". Need a complex type [STRUCT, ARRAY, MAP] but got "BINARY".

In [3]:
def execute_query(query: str):
    spark.sql(query).show(truncate=False)

In [4]:
# Kafka configurations
kafka_brokers = "localhost:9093"  # Update with your Kafka broker address
kafka_topic = "debezium.payment.terminals"
# Read data from Kafka (batch mode)
kafka_df = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_brokers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .load()

In [5]:
kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, LongType

# Define the schema for the JSON messages
column_schema = StructType([
StructField("schema", StringType(), False),
StructField("payload", StructType([
    StructField("before", StructType([
        StructField("terminal_id", IntegerType(), False),
        StructField("x_location", DoubleType(), False),
        StructField("y_location", DoubleType(), False)
    ]), True),
    StructField("after", StructType([
        StructField("terminal_id", IntegerType(), False),
        StructField("x_location", DoubleType(), False),
        StructField("y_location", DoubleType(), False)
    ]), True),
    StructField("source", StructType([
        StructField("version", StringType(), False),
        StructField("connector", StringType(), False),
        StructField("name", StringType(), False),
        StructField("ts_ms", LongType(), False),
        StructField("snapshot", StringType(), True),
        StructField("db", StringType(), False),
        StructField("sequence", StringType(), True),
        StructField("ts_us", LongType(), True),
        StructField("ts_ns", LongType(), True),
        StructField("schema", StringType(), False),
        StructField("table", StringType(), False),
        StructField("txId", LongType(), True),
        StructField("lsn", LongType(), True),
        StructField("xmin", LongType(), True)
    ]), False),
    StructField("transaction", StructType([
        StructField("id", StringType(), False),
        StructField("total_order", LongType(), False),
        StructField("data_collection_order", LongType(), False)
    ]), True),
    StructField("op", StringType(), False),
    StructField("ts_ms", LongType(), True),
    StructField("ts_us", LongType(), True),
    StructField("ts_ns", LongType(), True)
]))])

from pyspark.sql.functions import from_json

kafka_df = kafka_df.withColumn("value", col("value").cast(StringType())) \
    .withColumn("key", col("key").cast(StringType()))
# Apply the schema to the DataFrame
kafka_df = kafka_df.withColumn("value", from_json("value", column_schema))
kafka_df.createOrReplaceTempView("kafkadata1")

25/02/11 11:22:05 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [7]:
execute_query("select * from kafkadata1")

25/02/11 11:22:10 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/11 11:22:10 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/11 11:22:10 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/11 11:22:10 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
25/02/11 11:22:10 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
execute_query(""" create or replace temp view latest_data1 as 
              with extracted_data  as (
                    select value.payload.after.terminal_id,
                    value.payload.after.x_location, 
                    value.payload.after.y_location,
                    timestamp
                    from kafkadata1),
              ranked_data as (
                  select terminal_id,
                  x_location,
                  y_location, 
                  timestamp,
                  ROW_NUMBER() over (partition by terminal_id order by timestamp DESC) as rn
                  from extracted_data) 
              select terminal_id,
                  x_location,
                  y_location,
                  now() as row_created_timestamp,
                  now() as row_updated_timestamp
              from ranked_data 
              where rn=1""")

++
||
++
++



In [9]:
execute_query("CREATE DATABASE IF NOT EXISTS payment;")
execute_query("show databases")

++
||
++
++

+---------+
|namespace|
+---------+
|default  |
|payment  |
+---------+



In [10]:
execute_query("""
        CREATE TABLE IF NOT EXISTS payment.terminal (
      terminal_id INT,
      x_location FLOAT,
      y_location FLOAT,
      row_created_timestamp TimeStamp,
      row_updated_timestamp TimeStamp
)
USING iceberg
""")

++
||
++
++



In [11]:
execute_query("""
MERGE INTO payment.terminal AS target
USING latest_data1 AS source
ON target.terminal_id = source.terminal_id
WHEN MATCHED THEN
  UPDATE SET
    target.x_location = source.x_location,
    target.y_location = source.y_location,
    target.row_updated_timestamp = source.row_updated_timestamp
WHEN NOT MATCHED THEN
  INSERT (
    terminal_id,
    x_location,
    y_location,
    row_created_timestamp,
    row_updated_timestamp
  )
  VALUES (
    source.terminal_id,
    source.x_location,
    source.y_location,
    source.row_created_timestamp,
    source.row_updated_timestamp
  )
""")

25/02/11 11:22:40 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/11 11:22:40 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/11 11:22:40 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/11 11:22:40 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
25/02/11 11:22:40 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.
25/02/11 11:22:40 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/11 11:22:40 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/11 11:22:40 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/11 11:22:40 WARN AdminClientConfig: The c

++
||
++
++



In [12]:
execute_query("""select * from payment.terminal  """)

+-----------+----------+----------+--------------------------+--------------------------+
|terminal_id|x_location|y_location|row_created_timestamp     |row_updated_timestamp     |
+-----------+----------+----------+--------------------------+--------------------------+
|0          |54.88135  |71.518936 |2025-02-11 11:22:39.478597|2025-02-11 11:22:39.478597|
|1          |60.276337 |54.48832  |2025-02-11 11:22:39.478597|2025-02-11 11:22:39.478597|
|2          |42.36548  |64.58941  |2025-02-11 11:22:39.478597|2025-02-11 11:22:39.478597|
|3          |43.75872  |89.1773   |2025-02-11 11:22:39.478597|2025-02-11 11:22:39.478597|
|4          |96.36628  |38.34415  |2025-02-11 11:22:39.478597|2025-02-11 11:22:39.478597|
+-----------+----------+----------+--------------------------+--------------------------+



### StreamingJob

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, StructType, StructField

In [2]:
KAFKA_BROKERS = 'localhost:9093'
#KAFKA_TOPIC_NAME = "debezium.payment.terminals"
S3_ENDPOINT = "http://minio:9000"
CHECKPOINT_PATH = "s3a://commerce/checkpoints/debezium.payment.terminals/"

In [3]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.3.2")
                .config("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT) 
                .config("spark.hadoop.fs.s3a.access.key", "minio") 
                .config("spark.hadoop.fs.s3a.secret.key", "minio123") 
                .config("spark.hadoop.fs.s3a.path.style.access", "true") 
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
                .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
                .config("spark.sql.catalog.spark_catalog.type", "hadoop")
                .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://commerce/warehouse/") 
                .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") 
                .appName("streaming_sink_terminals")
         .getOrCreate())

:: loading settings :: url = jar:file:/Users/saurabh/Desktop/workspace/realtime_fraud_detection_system/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/saurabh/.ivy2/cache
The jars for the packages stored in: /Users/saurabh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dbfd3a17-6ab7-4c55-b40a-b643284da0dd;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in ce

In [4]:
# Read data from Kafka (batch mode)
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BROKERS) \
    .option("subscribe", "debezium.payment.terminals") \
    .option("startingOffsets", "earliest") \
    .load()

In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, LongType

# Define the schema for the JSON messages
column_schema = StructType([
StructField("schema", StringType(), False),
StructField("payload", StructType([
    StructField("before", StructType([
        StructField("terminal_id", IntegerType(), False),
        StructField("x_location", DoubleType(), False),
        StructField("y_location", DoubleType(), False)
    ]), True),
    StructField("after", StructType([
        StructField("terminal_id", IntegerType(), False),
        StructField("x_location", DoubleType(), False),
        StructField("y_location", DoubleType(), False)
    ]), True),
    StructField("source", StructType([
        StructField("version", StringType(), False),
        StructField("connector", StringType(), False),
        StructField("name", StringType(), False),
        StructField("ts_ms", LongType(), False),
        StructField("snapshot", StringType(), True),
        StructField("db", StringType(), False),
        StructField("sequence", StringType(), True),
        StructField("ts_us", LongType(), True),
        StructField("ts_ns", LongType(), True),
        StructField("schema", StringType(), False),
        StructField("table", StringType(), False),
        StructField("txId", LongType(), True),
        StructField("lsn", LongType(), True),
        StructField("xmin", LongType(), True)
    ]), False),
    StructField("transaction", StructType([
        StructField("id", StringType(), False),
        StructField("total_order", LongType(), False),
        StructField("data_collection_order", LongType(), False)
    ]), True),
    StructField("op", StringType(), False),
    StructField("ts_ms", LongType(), True),
    StructField("ts_us", LongType(), True),
    StructField("ts_ns", LongType(), True)
]))])

from pyspark.sql.functions import from_json

kafka_df = kafka_df.withColumn("value", col("value").cast(StringType())) \
    .withColumn("key", col("key").cast(StringType()))
# Apply the schema to the DataFrame
kafka_df = kafka_df.withColumn("value", from_json("value", column_schema))
kafka_df.createOrReplaceTempView("kafkadata")

25/02/12 19:53:51 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
spark.sql("""
create database if not exists payment
""").show(truncate=False)

++
||
++
++



In [7]:
spark.sql("""
        CREATE TABLE IF NOT EXISTS payment.terminal (
      terminal_id INT,
      x_location FLOAT,
      y_location FLOAT,
      row_created_timestamp TimeStamp,
      row_updated_timestamp TimeStamp
)
USING iceberg
""")

DataFrame[]

In [8]:
def foreach_batch_function(df, epoch_id):
    spark = df.sparkSession
    df.createOrReplaceTempView('micro_batch')
    latest_data = spark.sql(""" 
                with extracted_data  as (
                    select value.payload.after.terminal_id,
                    value.payload.after.x_location, 
                    value.payload.after.y_location,
                    timestamp
                    from micro_batch),
              ranked_data as (
                  select terminal_id,
                  x_location,
                  y_location, 
                  timestamp,
                  ROW_NUMBER() over (partition by terminal_id order by timestamp DESC) as rn
                  from extracted_data) 
              select terminal_id,
                  x_location,
                  y_location,
                  now() as row_created_timestamp,
                  now() as row_updated_timestamp
              from ranked_data 
              where rn=1""")
    latest_data.createOrReplaceTempView('latest_data')
    spark.sql("""
MERGE INTO payment.terminal AS target
USING latest_data AS source
ON target.terminal_id = source.terminal_id
WHEN MATCHED THEN
  UPDATE SET
    target.x_location = source.x_location,
    target.y_location = source.y_location,
    target.row_updated_timestamp = source.row_updated_timestamp
WHEN NOT MATCHED THEN
  INSERT (
    terminal_id,
    x_location,
    y_location,
    row_created_timestamp,
    row_updated_timestamp
  )
  VALUES (
    source.terminal_id,
    source.x_location,
    source.y_location,
    source.row_created_timestamp,
    source.row_updated_timestamp
  )
""").show(truncate=False)
    
query = kafka_df.writeStream \
.foreachBatch(foreach_batch_function) \
.option("checkpointLocation", CHECKPOINT_PATH) \
.trigger(processingTime='5 seconds') \
.start()

25/02/12 19:53:59 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


25/02/12 19:54:00 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/02/12 19:54:00 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/02/12 19:54:00 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/02/12 19:54:00 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
25/02/12 19:54:00 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


++
||
++
++



In [ ]:
query.awaitTermination()

25/02/12 20:43:36 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 266436 milliseconds
25/02/12 23:51:04 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1001 (localhost/127.0.0.1:9093) could not be established. Broker may not be available.
25/02/12 23:51:04 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1001 (localhost/127.0.0.1:9093) could not be established. Broker may not be available.
25/02/12 23:51:05 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1001 (localhost/127.0.0.1:9093) could not be established. Broker may not be available.
25/02/12 23:51:05 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1001 (localhost/127.0.0.1:9093) could not be established. Broker may not be available.
25/02/12 23:51:06 WARN NetworkClient: [AdminClient clientId=adminclient-1] Connection to node 1001 (localhost/127.0.0.1:909